In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-04 18:23:42,487 INFO: Initializing external client
2025-03-04 18:23:42,488 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:43,223 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [6]:
import sys
import subprocess

# Upgrade related dependencies
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "hsfs", "hopsworks", "protobuf"])

  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)


CompletedProcess(args=['/opt/anaconda3/bin/python', '-m', 'pip', 'install', '--upgrade', 'hsfs', 'hopsworks', 'protobuf'], returncode=0)

In [7]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.87s) 


In [8]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,238,73.0,2025-03-04 08:00:00+00:00
1,19,0.0,2025-03-04 08:00:00+00:00
2,226,2.0,2025-03-04 08:00:00+00:00
3,147,0.0,2025-03-04 08:00:00+00:00
4,214,0.0,2025-03-04 08:00:00+00:00
...,...,...,...
748,155,1.0,2025-03-05 00:00:00+00:00
749,189,0.0,2025-03-05 00:00:00+00:00
750,107,88.0,2025-03-05 00:00:00+00:00
751,88,10.0,2025-03-05 00:00:00+00:00


In [9]:
#a.info()

In [10]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 18:23:53,765 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:23:53,767 INFO: Initializing external client
2025-03-04 18:23:53,767 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:54,385 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Current UTC time: 2025-03-04 23:23:53.765048+00:00
Next hour: 2025-03-05 00:00:00+00:00
Found 251 records


In [11]:
now = datetime.now(timezone.utc)

In [12]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
502,171,0.0,2025-03-05 00:00:00+00:00
503,83,0.0,2025-03-05 00:00:00+00:00
504,213,0.0,2025-03-05 00:00:00+00:00
505,114,39.0,2025-03-05 00:00:00+00:00
506,242,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
748,155,1.0,2025-03-05 00:00:00+00:00
749,189,0.0,2025-03-05 00:00:00+00:00
750,107,88.0,2025-03-05 00:00:00+00:00
751,88,10.0,2025-03-05 00:00:00+00:00


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  753 non-null    int64                  
 1   predicted_demand    753 non-null    float64                
 2   pickup_hour         753 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int64(1)
memory usage: 17.8 KB


In [14]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [15]:
dt = current_date.ceil('h')


In [16]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 18:23:55,367 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:23:55,369 INFO: Initializing external client
2025-03-04 18:23:55,369 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:23:55,945 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 


In [17]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.95s) 


,pickup_location_id,predicted_demand,pickup_hour


In [18]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,238,73.0,2025-03-04 08:00:00+00:00
1,19,0.0,2025-03-04 08:00:00+00:00
2,226,2.0,2025-03-04 08:00:00+00:00
3,147,0.0,2025-03-04 08:00:00+00:00
4,214,0.0,2025-03-04 08:00:00+00:00
...,...,...,...
748,155,1.0,2025-03-05 00:00:00+00:00
749,189,0.0,2025-03-05 00:00:00+00:00
750,107,88.0,2025-03-05 00:00:00+00:00
751,88,10.0,2025-03-05 00:00:00+00:00


In [19]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [20]:
predictions = fetch_next_hour_predictions()

2025-03-04 18:24:15,949 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:24:15,951 INFO: Initializing external client
2025-03-04 18:24:15,952 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:24:16,510 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Current UTC time: 2025-03-04 23:24:15.949114+00:00
Next hour: 2025-03-05 00:00:00+00:00
Found 251 records


In [21]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-04 18:24:17,572 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:24:17,574 INFO: Initializing external client
2025-03-04 18:24:17,574 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:24:18,117 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [22]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.76s) 


In [23]:
df["pickup_hour"].max()

Timestamp('2025-03-05 00:00:00+0000', tz='Etc/UTC')

In [24]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 00:00:00+00:00


In [25]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
502,171,0.0,2025-03-05 00:00:00+00:00
503,83,0.0,2025-03-05 00:00:00+00:00
504,213,0.0,2025-03-05 00:00:00+00:00
505,114,39.0,2025-03-05 00:00:00+00:00
506,242,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
748,155,1.0,2025-03-05 00:00:00+00:00
749,189,0.0,2025-03-05 00:00:00+00:00
750,107,88.0,2025-03-05 00:00:00+00:00
751,88,10.0,2025-03-05 00:00:00+00:00
